## Preparation

In [0]:
# Import libraries and path define

import os
file_path = 'D:/Data/probe_match'
import numpy as np
import pandas as pd
import pickle
import math

# Read probe data
probe=pd.read_pickle(os.path.join(file_path, 'probe_in_window.pkl'))
#probe = pd.read_csv(os.path.join(file_path, 'Transformed_ProbePoints.csv'), names = ['sampleID', 'dateTime', 'sourceCode', 'latitude', 'longitude', 'altitude', 'speed', 'heading', 'X', 'Y'], header = 0)
print("probe shape: ", probe.shape)
#print(probe.head())

# Convert probe df into list, within target area

#id_count = 0
#probe_list = []
#for id in probe['sampleID'].unique():
#    probe_list.append(probe[probe['sampleID'] == id][['X', 'Y', 'altitude', 'speed', 'heading', 'sampleID']].values.tolist())
#    id_count += 1
#print("trajectory number: ", id_count)

# Read link data

link = pd.read_pickle(os.path.join(file_path, 'Link_in_window.pkl'))
#print(link.head())
print("link shape: ", link.shape)

probe shape:  (291674, 10)
trajectory number:  7332
link shape:  (13047, 18)


In [0]:
# Build link nodes df

nodes = []
nodes_highway = []
nodes_local = []
nodes_ref = []
nodes_nref = []

def addAllNodeList(alink):
    somenodes = alink.tolist()[-1]
    nodes_all = list([somenodes[i][0], somenodes[i][1], i, alink[0]] for i in range(len(somenodes)))
    nodes_ref = [somenodes[0][0], somenodes[0][1], alink[0], alink[1]]
    nodes_nref = [somenodes[-1][0], somenodes[-1][1], alink[0], alink[2]]
    category = alink[4]
    return [nodes_all, nodes_ref, nodes_nref, category]
for index, row in link.iterrows():
    [v_nodes_all, v_nodes_ref, v_nodes_nref, v_category] = addAllNodeList(row)
    nodes.extend(v_nodes_all)
    nodes_ref.append(v_nodes_ref)
    nodes_nref.append(v_nodes_nref)
    if v_category < 5:
        nodes_highway.extend(v_nodes_all)
    else:
        nodes_local.extend(v_nodes_all)
        
nodes = pd.DataFrame(nodes, columns = ['X', 'Y', 'sequence', 'link'])
nodes_highway = pd.DataFrame(nodes_highway, columns = ['X', 'Y', 'sequence', 'link'])
nodes_local = pd.DataFrame(nodes_local, columns = ['X', 'Y', 'sequence', 'link'])
nodes_ref = pd.DataFrame(nodes_ref, columns = ['X', 'Y', 'link', 'refNodeID'])
nodes_nref = pd.DataFrame(nodes_nref, columns = ['X', 'Y', 'link', 'nrefNodeID'])

## Initial candidates generation

In [0]:
# Global links search

def nearestNodes(x, y, length, candidate_list = nodes):
    return candidate_list[(candidate_list['X'] > (x - length))
                 & (candidate_list['X'] < (x + length))
                 & (candidate_list['Y'] > (y - length))
                 & (candidate_list['Y'] < (y + length))]['link'].unique().tolist()
def nearestMagic(x, y, local_threshold = 5, highway_threshold = 2):

    import math
       
    local_candidate_ID = nearestNodes(x, y, 100, nodes_local)
    if len(local_candidate_ID) == 0:
        local_result_nodes = []
    else:
        local_candidate = nodes_local[nodes_local['link'].isin(local_candidate_ID)]
        local_result_nodes = local_candidate.groupby('link').mean().apply(lambda row: math.sqrt((row['X'] - x)**2 + (row['Y'] - y)**2), axis = 1).nsmallest(local_threshold).index.tolist()
       
    
    highway_candidate_ID = nearestNodes(x, y, 500, nodes_highway)
    if len(highway_candidate_ID) == 0:
        highway_result_nodes = []
    else:
        highway_candidate = nodes_highway[nodes_highway['link'].isin(highway_candidate_ID)]
        highway_result_nodes = highway_candidate.groupby('link').mean().apply(lambda row: math.sqrt((row['X'] - x)**2 + (row['Y'] - y)**2), axis = 1).nsmallest(highway_threshold).index.tolist()
    
    return local_result_nodes + highway_result_nodes

#candidates = dict()
#for traj in probe_list:
#    candidates[int(traj[0][-1])] = nearestMagic(traj[0][0], traj[0][1])

# Write result to disk

#with open(os.path.join(file_path, 'candidates_hanover_local_highway.pkl'), 'wb') as fp:
#    pickle.dump(candidates, fp)

## Map matching

In [0]:
def delta_distance(probe_point,from_linkpoint,to_linkpoint):
    x1=probe_point[0]-from_linkpoint[0]
    y1=probe_point[1]-from_linkpoint[1]
    x2=to_linkpoint[0]-from_linkpoint[0]
    y2=to_linkpoint[1]-from_linkpoint[1]
    dot_product=x1*x2+y1*y2
    link_length=(x2**2+y2**2)**(1/2)
    projection=dot_product/link_length
    
    if projection >= 0:
        return max(projection-link_length,0)
    else:
        return abs(projection)
    
def straight_distance(probe_point,from_linkpoint,to_linkpoint):
    x1=probe_point[0]-from_linkpoint[0]
    y1=probe_point[1]-from_linkpoint[1]
    x2=to_linkpoint[0]-from_linkpoint[0]
    y2=to_linkpoint[1]-from_linkpoint[1]
    cross_product=x1*y2-x2*y1
    return abs(cross_product/(x2**2+y2**2)**(1/2))

distance_threshold =160
speed_threshold = 10
weight_heading = 20
weight_distance = 80

def tws(candidate_link,probepoints):
    # probe_speed,probe_heading,probe_(X,Y),link_ID
    linknodes = candidate_link.values.tolist()[0][-1]
    nodes_list = list([linknodes[i][0], linknodes[i][1]] for i in range(len(linknodes)))
    probe = [probepoints['X'],probepoints['Y']]
    
    # first calculate the distances of probe and link and f(D):
    # XXXXXXXXX 1. find the nearest 2, 3 or 4 points on link
    # XXXXXXXXX 2. compute the probe distance to line of the nearest 2 points and the area of triangle
    #
    # 3. if distance ==0 then f(D)=1
    # 3.1. if distance == threshold then f(D)=-1 # threshold in paper is 160m
    # XXXXXXXX 3.2. if the probe is not in range of line between nearest two point
    # XXXXXXXX 3.2.1. we will find the third point on link and find the D again
    # XXXXXXXX 3.2.2. take the D' comparing with D+\Delta D take the smaller one compute f(D)
        
    distance = distance_threshold
    link_vector =[0,0]
    for j in range(len(linknodes)-1):
        temp_distance= straight_distance(probe,nodes_list[j],nodes_list[j+1])+delta_distance(probe,nodes_list[j],nodes_list[j+1])
        if j==0:
            link_vector=[nodes_list[j+1][0]-nodes_list[j][0],nodes_list[j+1][1]-nodes_list[j][1]]
        if temp_distance < distance :
            distance = temp_distance
            link_vector = [nodes_list[j+1][0]-nodes_list[j][0],nodes_list[j+1][1]-nodes_list[j][1]]

    
    f_D = (distance_threshold/2-distance)/(distance_threshold/2)
        
    # second f(\theta)
    # 1. \theta denote the angle difference between the vehicle heading and link direction with respect to the north
    # 1.?. our heading is not good enough when speed is small. how to avoid error for this situations?
    # 1.solution just give a speed cutoff here for speed weight
    # 2. f(\theta)=cos(\theta)
    
    probe_heading = probepoints['heading']
    probe_speed = probepoints['speed']
    f_theta = 2*abs(link_vector[0]*math.sin(math.radians(probe_heading))+link_vector[1]*math.cos(math.radians(probe_heading)))/((link_vector[0]**2+link_vector[1]**2 )**(1/2))*(1 if probe_speed>speed_threshold else 0) -1
    
    
    tw_score= weight_distance*f_D + weight_heading*f_theta
    
    
    
    
    return tw_score

def map_matching(probe_id):
    id=probe_id
    temp = probe[probe.sampleID==id]#.copy().reset_index()
    inits = initial_candidate.get(id)
    ls=[]
    current_candidate_id=0
    candidate_id = inits
    output=[]
    for num in range(temp.shape[0]):
        temp_score = -100
        matching_link = None
        if len(candidate_id) == 0:
            ls.append(np.nan)
            continue
        for link_id in candidate_id:
            temp_link =link[link.linkPVID==link_id]
            if tws(temp_link,temp.iloc[num]) > temp_score:
                current_candidate_id=link_id
            #print(link_id)
                temp_score = tws(temp_link,temp.iloc[num])
            else:
                continue
        ls.append(current_candidate_id)
        row = link[link.linkPVID==current_candidate_id]
        candidate_id=link.iloc[np.where(((link.refNodeID.values==row.refNodeID.values) 
           | (link.nrefNodeID.values==row.refNodeID.values) 
           | (link.refNodeID.values==row.nrefNodeID.values) 
           | (link.nrefNodeID.values==row.nrefNodeID.values)))]['linkPVID'].values
    output=temp[['X','Y','altitude']].copy()
    output['linkID']=ls
    return(output)

In [0]:
probe=pd.read_pickle(os.path.join(file_path, 'probe_in_window.pkl'))
link = pd.read_pickle(os.path.join(file_path, 'Link_in_window.pkl'))
initial_candidate = pickle.load(open(os.path.join(file_path, 'candidates_hanover_local_highway.pkl'), 'rb' ))
#probe=probe.reset_index()
#link= link.reset_index()
test_match = map_matching(15460)
print(test_match)

                 X              Y  altitude     linkID
8344 -56376.838055  150198.669700       100   51925966
8345 -56334.660206  150222.735874       104   51925966
8346 -56328.764786  150228.755845       104   51925966
8347 -56327.354190  150228.823859       104   51925966
8348 -56326.929768  150229.005494       104   51925966
8349 -56326.689965  150229.012117       105   51925966
8350 -56326.494876  150229.103127       105   51925966
8351 -56304.608901  150234.374695       106   51925932
8352 -56251.908040  150257.801278       107   51925925
8353 -56196.446414  150313.148194       107   51925925
8354 -56165.931876  150394.136709       106  723473317
8355 -56157.150439  150490.580286       106  723473320
8356 -56146.117253  150566.456196       107  723473320
8357 -56138.883946  150603.787723       107  723473320
8358 -56139.335174  150607.828577       106  723473320
8359 -56138.411470  150609.784853       106  723473320
8360 -56131.910935  150654.562407       104  723473319
8361 -5611

## Generate dataset of height and distance from ref node from link dataset

In [0]:
probe=pd.read_pickle(os.path.join(file_path, 'probe_in_window.pkl'))
link = pd.read_pickle(os.path.join(file_path, 'Link_in_window.pkl'))
link_valid = link[link.apply(lambda row: len(row['slopeInfo']), axis = 1) != 0].values.tolist()
slope_from_link_list = []
for row in link_valid:
    for i in range(len(row[14])):
        slope_from_link_list.append([row[0], row[16][i][0], row[14][i][2], row[16][i][1]])
slope_from_link = pd.DataFrame.from_records(slope_from_link_list, columns=['linkPVID', 'distFromRef', 'height', 'slope'])
#with open(os.path.join(file_path, 'slope_from_link.pkl'), 'wb') as fp:
#    pickle.dump(slope_from_link, fp)

## Generate dataset of probe height and distance to ref node

In [0]:
%%time
def probe_to_ref(p, fit_ID):
    x = p[0]
    y = p[1]
    fit = nodes[nodes['link'] == fit_ID]
    #print(fit_ID)
    base_points = fit.loc[fit.apply(lambda row: math.sqrt((row['X'] - x)**2 + (row['Y'] - y)**2), axis = 1).nsmallest(2).index]['sequence'].sort_values().index
    b1 = base_points[0]
    b2 = base_points[1]
    def two_nodes_dist(b1, b2):
        x1 = nodes.loc[b1]['X']
        x2 = nodes.loc[b2]['X']
        y1 = nodes.loc[b1]['Y']
        y2 = nodes.loc[b2]['Y']
        import math
        return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)
    def probe_node_dist(x, y, b):
        x2 = nodes.loc[b]['X']
        y2 = nodes.loc[b]['Y']
        return math.sqrt((x - x2)**2 + (y - y2)**2)
    def node_ref_dist(b):
        dist_sum = 0
        for i in range(int(fit.loc[b]['sequence'])):
            dist_sum += two_nodes_dist(fit[fit['sequence'] == i].index[0], fit[fit['sequence'] == (i+1)].index[0])
        return dist_sum
    def vector_2(point1, point2):
        return [point2[0] - point1[0], point2[1] - point1[1]]
    if (math.sqrt((nodes.loc[b1]['X'] - x)**2 + (nodes.loc[b1]['Y'] - y)**2) + math.sqrt((nodes.loc[b2]['X'] - x)**2 + (nodes.loc[b2]['Y'] - y)**2))> (3 * two_nodes_dist(b1, b2)):
        return np.nan
    return node_ref_dist(b1) + np.inner(vector_2(nodes.loc[b1][['X', 'Y']], [x, y]), vector_2(nodes.loc[b2][['X', 'Y']], [x, y])) / two_nodes_dist(b1, b2)
slope_from_probe = []
count = 0
check_list = slope_from_link['linkPVID'].unique().tolist()
probe=pd.read_pickle(os.path.join(file_path, 'probe_in_window.pkl'))
link = pd.read_pickle(os.path.join(file_path, 'Link_in_window.pkl'))
initial_candidate = pickle.load(open(os.path.join(file_path, 'candidates_hanover_local_highway.pkl'), 'rb' ))
iterator = probe['sampleID'].unique().tolist()

Wall time: 121 ms


In [0]:
%%time
for sampleID in iterator[501:1000]:
    #probe=pd.read_pickle(os.path.join(file_path, 'probe_in_window.pkl'))
    #link = pd.read_pickle(os.path.join(file_path, 'Link_in_window.pkl'))
    #initial_candidate = pickle.load(open(os.path.join(file_path, 'candidates_hanover_local_highway.pkl'), 'rb' ))
    #probe=probe.reset_index()
    #link= link.reset_index()
    test_match = map_matching(sampleID)
    for row in test_match.values.tolist():
        if math.isnan(row[3]):
            continue
        if not row[3] in check_list:
            continue
        slope_temp = probe_to_ref(row[:2], row[3])
        if not math.isnan(slope_temp):
            slope_from_probe.append([row[3], slope_temp, row[2]])
    count += 1
    if count > 1000:
        break
#print(slope_from_probe)

Wall time: 2min 46s


### Try parallel

%%time
from joblib import Parallel, delayed
import multiprocessing
slope_from_probe = []
count = 0
check_list = slope_from_link['linkPVID'].unique().tolist()
probe=pd.read_pickle(os.path.join(file_path, 'probe_in_window.pkl'))
link = pd.read_pickle(os.path.join(file_path, 'Link_in_window.pkl'))
initial_candidate = pickle.load(open(os.path.join(file_path, 'candidates_hanover_local_highway.pkl'), 'rb' ))
for sampleID in probe['sampleID'].unique().tolist():
    #probe=pd.read_pickle(os.path.join(file_path, 'probe_in_window.pkl'))
    #link = pd.read_pickle(os.path.join(file_path, 'Link_in_window.pkl'))
    #initial_candidate = pickle.load(open(os.path.join(file_path, 'candidates_hanover_local_highway.pkl'), 'rb' ))
    #probe=probe.reset_index()
    #link= link.reset_index()
    test_match = map_matching(sampleID)
    inputs = test_match.values.tolist()
    def processInput(row):
        if math.isnan(row[3]):
            return np.nan
        if row[3] in check_list:
            return np.nan
        slope_temp = probe_to_ref(row[:2], row[3])
        return [row[3], slope_temp, row[2]]
    num_cores = multiprocessing.cpu_count()
    slope_from_probe.append(Parallel(n_jobs=num_cores)(delayed(processInput)(row) for row in inputs))
    count += 1
    if count > 50:
        break

In [0]:
with open(os.path.join(file_path, 'slope_from_probe.pkl'), 'wb') as fp:
    pickle.dump(pd.DataFrame.from_records(slope_from_probe, columns=['linkPVID', 'distFromRef', 'height']), fp)

In [0]:
pd.DataFrame.from_records(slope_from_probe, columns=['linkPVID', 'distFromRef', 'height'])

,linkPVID,distFromRef,height
0,51921783.0,63.552500,100.0
1,708470043.0,106.562027,102.0
2,708470042.0,568.199436,102.0
3,708470042.0,287.104942,99.0
4,708470042.0,-17.804264,96.0
5,724904578.0,-4.853254,95.0
6,724712669.0,-3.695132,96.0
7,540641096.0,-105.203617,96.0
8,540641096.0,-48.562653,96.0
9,540641094.0,-62.839271,96.0


## Visulization

In [0]:
import pyproj as proj
import gmaps
import ipywidgets as widgets
gmaps.configure()


# GPS coordinates to CRS coordinates

crs_wgs = proj.Proj(init='epsg:4326')
crs_bng = proj.Proj(init='epsg:5243')

def wgs2XY(lon, lat):
    x, y = proj.transform(crs_wgs, crs_bng, lon, lat)
    return (x, y)

def XY2wgs(x, y):
    lon, lat = proj.transform(crs_bng, crs_wgs, x, y)
    return (lat, lon)

# Preparation for gmaps

def coordMaker(traj):
    lst_lat = [float(i[0]) for i in traj]
    lst_lon = [float(i[1]) for i in traj]
    return XY2wgs(sum(lst_lat)/len(lst_lat), sum(lst_lon)/len(lst_lon))

def traj_to_df(traj):
    labels = ['X', 'Y', 'altitude', 'speed', 'heading', 'sampleID']
    return pd.DataFrame.from_records(traj, columns=labels)

def lines(link, color = 'red'):
    nodes_list = link.apply(lambda row: (row['X'], row['Y']), axis = 1).values
    features = []
    for i in range(len(nodes_list) - 1):
        features.append(gmaps.Line(start=XY2wgs(nodes_list[i][0], nodes_list[i][1]), end=XY2wgs(nodes_list[i+1][0], nodes_list[i+1][1]), stroke_weight=3.0, stroke_color=color))
    return features

def draw_link_line(linkID):
    try:
        link = nodes[nodes['link'] == linkID]
        features = lines(link)
    except:
        features = []
        for alinkID in linkID:
            link = nodes[nodes['link'] == alinkID]
            features += lines(link)
    fig = gmaps.figure(center=XY2wgs(link['X'].mean(), link['Y'].mean()), zoom_level=14, map_type='TERRAIN')
    drawing = gmaps.drawing_layer(features=features)
    fig.add_layer(drawing)
    return fig

def draw_candidate(traj):
    if len(traj) > 50:
        zoom = 13
    else:
        zoom = 14
    x = traj[0][0]
    y = traj[0][1]
    lat, lon = XY2wgs(x, y)
    linkID = nearestMagic(x, y)
    try:
        link = nodes[nodes['link'] == linkID]
        features = lines(link)
    except:
        features = []
        for alinkID in linkID:
            link = nodes[nodes['link'] == alinkID]
            features += lines(link)
    features.append(gmaps.Marker((lat, lon)))
    features += lines(traj_to_df(traj), color='green')
    fig = gmaps.figure(center=(lat, lon), zoom_level=14, map_type='TERRAIN')
    drawing = gmaps.drawing_layer(features=features)
    fig.add_layer(drawing)
    return fig

def draw_match(sampleID):
    traj = probe[probe['sampleID'] == sampleID]
    link = map_matching(sampleID)
    linkID = link['linkID'].unique()
    if len(linkID) == 1:
        features = lines(link)
    else:
        features = []
        for alinkID in linkID:
            alink = nodes[nodes['link'] == alinkID]
            features += lines(alink)
    features += lines(traj, color = 'green')
    x = traj.groupby('sampleID').mean()['X'].values.tolist()[0]
    y = traj.groupby('sampleID').mean()['Y'].values.tolist()[0]
    lat, lon = XY2wgs(x, y)
    fig = gmaps.figure(center=(lat, lon), zoom_level=14, map_type='TERRAIN')
    drawing = gmaps.drawing_layer(features=features)
    fig.add_layer(drawing)
    return fig

In [0]:
# Make some test

test_traj = probe_list[66]
test_link = nearestMagic(test_traj[0][0], test_traj[0][1])

In [0]:
# Draw some links

draw_link_line(554712478)

In [0]:
# Draw some matched candidates

draw_candidate(test_traj)

In [0]:
# Draw a matched result

probe=pd.read_pickle(os.path.join(file_path, 'probe_in_window.pkl'))
link = pd.read_pickle(os.path.join(file_path, 'Link_in_window.pkl'))
initial_candidate = pickle.load(open(os.path.join(file_path, 'candidates_hanover_local_highway.pkl'), 'rb' ))
probe=probe.reset_index()
link= link.reset_index()
import random
draw_match(random.sample(probe['sampleID'].unique().tolist(), 1)[0])

In [0]:
draw_match(probe['sampleID'].unique().tolist()[9])

In [0]:
probe['sampleID'].unique().tolist()[9]